In [ ]:
from google.colab import drive
from google.cloud import bigquery
import pandas as pd
import numpy as np
import glob
from sklearn.model_selection import train_test_split
import re
from datetime import datetime

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/My Drive/Capstone/

/content/drive/My Drive/Capstone


# Clean Datasets (3): Austin Intake, Austin Outcome, Sonoma Intake and Outcome
## More than likely both animal shelter districts are using the same or similar data managment systems because all three datasets are similar in information and format. However, small adjustments were needed before combining all three.

### **The following fields are removed because they don't reflect in both Austin and Sonoma**. However, could be beneficial if one dataset is used.


*   **Sex Upon outcome**: Only captured in Austin dataset, but useful in understanding spay / neutering practices in animal shelter
*   **Kennel id & impound number**: Only captured in Sonoma dataset, but could be useful to understand relationship between kennel selection and outcome. For example: distance from animals with aggressision could have an impact on a non aggressive animal's behavior overtime.
*   **Outcome condition**: Only captured in Sonoma dataset, but useful to compare condition with intake. Alternative column that is present in both can be outcome subtype
*   **Intake Jurisdiction/Outcome Jurisdiction/Outcome Zip Code/Location**: Location related columns were only captured in Sonoma dataset
*   **Size**: Only captured in sonoma dataset but could be enriched with a breed to size publicly available dataset, if we wish to pursue training with the combined dataset





In [ ]:
##clean austin intake csv

##load cvs
austin_intake =  pd.read_csv('Austin_Animal_Center_Intakes_20240609.csv')

#convert to upper case
austin_intake['Name'] = austin_intake['Name'].str.upper()
austin_intake['Breed'] = austin_intake['Breed'].str.upper()
austin_intake['Color'] = austin_intake['Color'].str.upper()
austin_intake['Type'] = austin_intake['Animal Type'].str.upper()
austin_intake['Intake Type'] = austin_intake['Intake Type'].str.upper()

#align sex values with sonoma dataset
austin_intake['Sex'] = np.where(austin_intake['Sex upon Intake'].str.contains('Neutered'), 'NEUTERED', austin_intake['Sex upon Intake'])
austin_intake['Sex'] = np.where(austin_intake['Sex'].str.contains('Spayed'), 'SPAYED', austin_intake['Sex'])
austin_intake['Sex'] = np.where(austin_intake['Sex'] == 'Intact Male', 'MALE', austin_intake['Sex'])
austin_intake['Sex'] = np.where(austin_intake['Sex'] == 'Intact Female', 'FEMALE', austin_intake['Sex'])

#align condition values with sonoma dataset
healthy = ['NORMAL', 'AGED','PREGNANT', 'SPACE']
treatable = ['MED ATTN','SICK', 'INJURED','NURSING','BEHAVIOR','MEDICAL']
untreatable = ['AGONAL','FERAL','MED URGENT','CONGENITAL','NEUROLOGIC', 'PANLEUK', 'PARVO']
unknown = ['OTHER','UNKNOWN']

austin_intake['Intake Condition'] = np.where(austin_intake['Intake Condition'].str.upper().isin(healthy),'HEALTHY',austin_intake['Intake Condition'])
austin_intake['Intake Condition'] = np.where(austin_intake['Intake Condition'].str.upper().isin(treatable),'TREATABLE', austin_intake['Intake Condition'])
austin_intake['Intake Condition'] = np.where(austin_intake['Intake Condition'].str.upper().isin(untreatable),'UNTREATABLE', austin_intake['Intake Condition'])
austin_intake['Intake Condition'] = np.where(austin_intake['Intake Condition'].str.upper().isin(unknown),'Unknown', austin_intake['Intake Condition'])

#convert to datetime
austin_intake['Intake Date'] = pd.to_datetime(austin_intake['DateTime'], errors='coerce')

#add intake counter to align with sonoma dataset
austin_intake['Count'] = 1


In [ ]:
##Keep only intake columns that align with sonoma dataset

austin_intake.head()
austin_intake = austin_intake[['Name','Type','Breed','Color','Sex','Intake Date','Animal ID','Intake Condition', 'Count']]
austin_intake.head()

,Name,Type,Breed,Color,Sex,Intake Date,Animal ID,Intake Condition,Count
0,NaN,CAT,DOMESTIC SHORTHAIR MIX,CALICO,FEMALE,2013-10-21 07:59:00,A665644,TREATABLE,1
1,*ALANA,CAT,DOMESTIC MEDIUM HAIR MIX,BLACK,FEMALE,2013-10-22 11:11:00,A665739,HEALTHY,1
2,NaN,DOG,CAIRN TERRIER MIX,TAN/WHITE,MALE,2013-10-22 15:10:00,A665763,HEALTHY,1
3,DISCIPLE,DOG,PIT BULL,BLACK,MALE,2013-10-23 11:42:00,A379998,HEALTHY,1
4,OTTER,DOG,NORFOLK TERRIER MIX,TAN,SPAYED,2013-10-01 14:49:00,A634503,HEALTHY,1


In [ ]:
##Clean austin outcome csv

#load csv
austin_outcome =  pd.read_csv('Austin_Animal_Center_Outcomes_20240609.csv')

#convert to upper case
austin_outcome['Name'] = austin_outcome['Name'].str.upper()
austin_outcome['Breed'] = austin_outcome['Breed'].str.upper()
austin_outcome['Color'] = austin_outcome['Color'].str.upper()
austin_outcome['Type'] = austin_outcome['Animal Type'].str.upper()
austin_outcome['Outcome Type'] = austin_outcome['Outcome Type'].str.upper()

#align sex values with sonoma dataset
austin_outcome['Sex Upon Outcome'] = np.where(austin_outcome['Sex upon Outcome'].str.contains('Neutered'), 'NEUTERED', austin_outcome['Sex upon Outcome'])
austin_outcome['Sex Upon Outcome'] = np.where(austin_outcome['Sex Upon Outcome'].str.contains('Spayed'), 'SPAYED', austin_outcome['Sex Upon Outcome'])
austin_outcome['Sex Upon Outcome'] = np.where(austin_outcome['Sex Upon Outcome'] == 'Intact Male', 'MALE', austin_outcome['Sex Upon Outcome'])
austin_outcome['Sex Upon Outcome'] = np.where(austin_outcome['Sex Upon Outcome'] == 'Intact Female', 'FEMALE', austin_outcome['Sex Upon Outcome'])

#align outcome type with sonoma dataset
austin_outcome['Outcome Type'] = np.where(austin_outcome['Outcome Type'].str.contains('RELOCATE'), 'TRANSFER', austin_outcome['Outcome Type'])
austin_outcome['Outcome Type'] = np.where(austin_outcome['Outcome Type'].str.contains('RTO'), 'RTOS', austin_outcome['Outcome Type'])
austin_outcome['Outcome Type'] = np.where(austin_outcome['Outcome Type'].str.contains('MISSING'), 'LOST', austin_outcome['Outcome Type'])
austin_outcome['Outcome Type'] = np.where(austin_outcome['Outcome Type'].str.contains('EUTHANASIA'), 'EUTHANIZE', austin_outcome['Outcome Type'])
austin_outcome['Outcome Type'] = np.where(austin_outcome['Outcome Type'].str.contains('DIED'), 'DISPOSAL', austin_outcome['Outcome Type'])

#convert to datetime
austin_outcome['Outcome Date'] = pd.to_datetime(austin_outcome['DateTime'], errors='coerce')

<ipython-input-35-3a43e82d5861>:27: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  austin_outcome['Outcome Date'] = pd.to_datetime(austin_outcome['DateTime'], errors='coerce')


In [ ]:
##Keep only outcome columns that align with sonoma dataset pt 2

austin_outcome.head()
austin_outcome = austin_outcome[['Name','Type', 'Breed','Color','Date of Birth','Animal ID','Outcome Date','Outcome Type','Outcome Subtype']]
austin_outcome.head()

,Name,Type,Breed,Color,Date of Birth,Animal ID,Outcome Date,Outcome Type,Outcome Subtype
0,CHUNK,CAT,DOMESTIC SHORTHAIR MIX,BROWN TABBY/WHITE,05/02/2017,A794011,2019-05-08 18:20:00,RTOS,NaN
1,GIZMO,DOG,CHIHUAHUA SHORTHAIR MIX,WHITE/BROWN,07/12/2017,A776359,2018-07-18 16:02:00,ADOPTION,NaN
2,NaN,OTHER,RACCOON,GRAY,08/16/2019,A821648,2020-08-16 11:38:00,EUTHANIZE,NaN
3,MOOSE,DOG,ANATOL SHEPHERD/LABRADOR RETRIEVER,BUFF,10/08/2015,A720371,2016-02-13 17:59:00,ADOPTION,NaN
4,NaN,CAT,DOMESTIC SHORTHAIR MIX,ORANGE TABBY,03/12/2014,A674754,2014-03-18 11:47:00,TRANSFER,Partner


In [ ]:
## Combine intake and outtake datasets

austin_df = pd.merge(austin_intake, austin_outcome, how="left", on=['Name','Type', 'Breed','Color','Animal ID'])

##Add Days in shelter column

austin_df['Days in Shelter'] = (austin_df['Outcome Date'] - austin_df['Intake Date']).dt.days

## Add source column

austin_df['Source'] = 'AUSTIN'

austin_df.head()

,Name,Type,Breed,Color,Sex,Intake Date,Animal ID,Intake Condition,Count,Date of Birth,Outcome Date,Outcome Type,Outcome Subtype,Days in Shelter,Source
0,NaN,CAT,DOMESTIC SHORTHAIR MIX,CALICO,FEMALE,2013-10-21 07:59:00,A665644,TREATABLE,1,09/21/2013,2013-10-21 11:39:00,TRANSFER,Partner,0.0,AUSTIN
1,*ALANA,CAT,DOMESTIC MEDIUM HAIR MIX,BLACK,FEMALE,2013-10-22 11:11:00,A665739,HEALTHY,1,09/16/2013,2013-12-20 17:36:00,ADOPTION,Foster,59.0,AUSTIN
2,NaN,DOG,CAIRN TERRIER MIX,TAN/WHITE,MALE,2013-10-22 15:10:00,A665763,HEALTHY,1,06/22/2013,2013-10-26 18:32:00,ADOPTION,NaN,4.0,AUSTIN
3,DISCIPLE,DOG,PIT BULL,BLACK,MALE,2013-10-23 11:42:00,A379998,HEALTHY,1,02/21/2003,2013-10-25 12:53:00,RETURN TO OWNER,NaN,2.0,AUSTIN
4,OTTER,DOG,NORFOLK TERRIER MIX,TAN,SPAYED,2013-10-01 14:49:00,A634503,HEALTHY,1,08/11/2011,2013-10-02 12:40:00,ADOPTION,NaN,0.0,AUSTIN


In [ ]:
## Clean sonoma dataset

#load csv
sonoma_df =  pd.read_csv('Animal_Shelter_Intake_and_Outcome_20240517.csv')

# Convert 'Date Of Birth' to datetime after reading the CSV df['Date Of Birth'] = pd.to_datetime(df['Date Of Birth'], errors='coerce')
sonoma_df['Date of Birth'] = pd.to_datetime(sonoma_df['Date Of Birth'].fillna('01/01/1900', inplace=True), errors='coerce')

#convert to datetime
sonoma_df['Intake Date'] = pd.to_datetime(sonoma_df['Intake Date'], errors='coerce')
sonoma_df['Outcome Date'] = pd.to_datetime(sonoma_df['Outcome Date'], errors='coerce')

#convert to uppercase
sonoma_df['Sex'] = sonoma_df['Sex'].str.upper()

#add source
sonoma_df['Source'] = 'SONOMA'

#Keep only columns that align with Austin
sonoma_df = sonoma_df[['Name','Type','Breed','Color','Sex','Date of Birth','Intake Date','Outcome Date','Days in Shelter','Animal ID','Outcome Type','Outcome Subtype','Intake Condition','Count','Source']]

sonoma_df.head()


,Name,Type,Breed,Color,Sex,Date of Birth,Intake Date,Outcome Date,Days in Shelter,Animal ID,Outcome Type,Outcome Subtype,Intake Condition,Count,Source
0,NaN,OTHER,BAT,BROWN,UNKNOWN,None,2024-03-16,2024-03-24,8,A425283,TRANSFER,LAB FRA,UNKNOWN,1,SONOMA
1,NaN,DOG,SIBERIAN HUSKY,BLACK/WHITE,MALE,None,2023-12-04,2024-01-08,35,A422697,EUTHANIZE,AGGRESSIVE,UNKNOWN,1,SONOMA
2,PINTO,DOG,PIT BULL,BR BRINDLE/WHITE,NEUTERED,None,2019-07-16,2019-08-30,45,A389457,TRANSFER,HSSC,HEALTHY,1,SONOMA
3,NaN,DOG,PIT BULL,GRAY,FEMALE,None,2023-11-28,2024-01-08,41,A422443,EUTHANIZE,BEH SEVERE,UNKNOWN,1,SONOMA
4,NaN,DOG,GERM SHEPHERD,TAN/BLACK,FEMALE,None,2023-11-11,2024-01-08,58,A422038,EUTHANIZE,AGGRESSIVE,UNKNOWN,1,SONOMA


In [ ]:
## Combine sonoma and austin
df = pd.concat([austin_df,sonoma_df])

df.head(100)

,Name,Type,Breed,Color,Sex,Intake Date,Animal ID,Intake Condition,Count,Date of Birth,Outcome Date,Outcome Type,Outcome Subtype,Days in Shelter,Source
0,NaN,CAT,DOMESTIC SHORTHAIR MIX,CALICO,FEMALE,2013-10-21 07:59:00,A665644,TREATABLE,1,09/21/2013,2013-10-21 11:39:00,TRANSFER,Partner,0.0,AUSTIN
1,*ALANA,CAT,DOMESTIC MEDIUM HAIR MIX,BLACK,FEMALE,2013-10-22 11:11:00,A665739,HEALTHY,1,09/16/2013,2013-12-20 17:36:00,ADOPTION,Foster,59.0,AUSTIN
2,NaN,DOG,CAIRN TERRIER MIX,TAN/WHITE,MALE,2013-10-22 15:10:00,A665763,HEALTHY,1,06/22/2013,2013-10-26 18:32:00,ADOPTION,NaN,4.0,AUSTIN
3,DISCIPLE,DOG,PIT BULL,BLACK,MALE,2013-10-23 11:42:00,A379998,HEALTHY,1,02/21/2003,2013-10-25 12:53:00,RETURN TO OWNER,NaN,2.0,AUSTIN
4,OTTER,DOG,NORFOLK TERRIER MIX,TAN,SPAYED,2013-10-01 14:49:00,A634503,HEALTHY,1,08/11/2011,2013-10-02 12:40:00,ADOPTION,NaN,0.0,AUSTIN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,PHOEBE,DOG,PIT BULL MIX,BLUE/WHITE,FEMALE,2013-10-05 16:17:00,A664597,Unknown,1,06/05/2013,2013-10-11 14:00:00,TRANSFER,Partner,5.0,AUSTIN
96,TWILIGHT,DOG,CHIHUAHUA SHORTHAIR MIX,WHITE/BROWN,SPAYED,2013-10-05 14:10:00,A664572,HEALTHY,1,10/05/2011,2013-10-09 13:13:00,TRANSFER,Partner,3.0,AUSTIN
97,*HAWKEYE,CAT,DOMESTIC SHORTHAIR MIX,WHITE/BLACK,MALE,2013-10-17 12:26:00,A665395,HEALTHY,1,04/17/2013,2013-10-29 12:15:00,TRANSFER,Partner,11.0,AUSTIN
98,SCOUT,DOG,TREEING WALKER COONHOUND MIX,WHITE/BLACK,FEMALE,2013-10-07 11:05:00,A664662,HEALTHY,1,08/13/2013,2013-10-13 13:41:00,ADOPTION,NaN,6.0,AUSTIN


# EDA Combined Dataset

# Apply util.py processing

In [ ]:
df.to_csv('sonoma_austin_combined.csv')

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

def load_df(params, name = 'sonoma_austin_combined.csv'):
    '''
    this loads the csv, cleans the data and creates new columns

    name is the name of the csv, this should be a direct or relative path to the csv database for Animal_Shelter_Intake_and_Outcome

    params options

    na_data will fill missing data with 'unknown', delete missing data or do nothing
    input options are...
        * 'fill'
        * 'drop'
        * False

    drop_outlier_days removes pets who have a lenght of stay exceeding the value YOU enter
    input options are...
        * False
        * or any integer

    embed creates 50x1 embedding vectors for color and breed.
        download https://nlp.stanford.edu/data/glove.6B.zip, unzip and save in repo
        * True
        * False

    sample_dict controls stratified sampling
        * stratify_col: a column name used for stratified sampling... spelling and caps must be exact
        * train_size: a fraction of data you want for the training data
        * validate_size: a fraction of data you want for the validate data
        * test_size: a fraction of data you want for the test data

    num_buckets how many buckets to break up length of stay into for model training
        creates new column Days_in_Shelter_Label
        * input is a integer
    '''

    df =  pd.read_csv(name)
    df['Date of Birth'].fillna('01/01/1900', inplace=True)
    # Convert 'Date Of Birth' to datetime after reading the CSV df['Date Of Birth'] = pd.to_datetime(df['Date Of Birth'], errors='coerce')
    df['Date Of Birth'] = pd.to_datetime(df['Date of Birth'], errors='coerce')
    df['Intake Date'] = pd.to_datetime(df['Intake Date'], errors='coerce')
    df['Outcome Date'] = pd.to_datetime(df['Outcome Date'], errors='coerce')
    df = clean_df(df, params)
    df = feature_eng(df)

    if params['embed']:
        # download https://nlp.stanford.edu/data/glove.6B.zip, unzip and save in repo
        glove_file_path = 'glove.6B.50d.txt'
        embeddings_index = load_glove_embeddings(glove_file_path)
        df = embed_colors(df, embeddings_index)
        df = embed_breeds(df, embeddings_index)


    num_buckets = params['num_buckets']
    class_labels = [i for i in range(num_buckets)]
    df['Days_in_Shelter_Label'], bin_edges = pd.qcut(df['Days_in_Shelter'], q=num_buckets, labels=class_labels, retbins=True)
    train_df, validate_df, test_df = train_validate_test_split(df, params)
    return train_df, validate_df, test_df

def feature_eng(df):
    '''this is where we add more columns'''

    # identify return animals, add new col for this
    df['Multiple_Visit_Count'] = df.groupby('Animal_ID')['Animal_ID'].transform('count')

    # calculate age at time of adoption
    df['Age_inDays_at_Outcome'] = (df['Outcome_Date'] - df['Date_Of_Birth']).dt.days

    # Create age groups
    bins = [0, 1, 3, 10, 40, 1000]  # Example bins for age groups
    bins_days = [day*365 for day in bins]
    labels = ['Puppy/Kitten', 'Young', 'Adult', 'Senior', 'Unknown']
    df['Age_Group'] = pd.cut(df['Age_inDays_at_Outcome'], bins=bins_days, labels=labels)
    # df['Age_Group'].fillna('Unknown', inplace=True)

    # Example of feature interaction
    df['Is_Aggressive'] = df['Outcome_Subtype'].apply(lambda x: 1 if 'AGGRESSIVE' in str(x) else 0)

    # 1 if animal came to shelter with name, 0 f shelter named animal
    df['Has_Name'] = df['Name'].apply(lambda x: 0 if '*' in str(x) else 1)

    # 1 if animal is fixed, else 0
    df['Is_Fixed'] = df.Sex.apply(lambda x: 1 if 'NEUTERED' in str(x) or 'SPAYED' in str(x) else 0)
    # animal gender ignoring fix status
    df['Sex'] = df.Sex.apply(lambda x: 'MALE' if 'NEUTERED' in str(x) else str(x))
    df['Sex'] = df.Sex.apply(lambda x: 'FEMALE' if 'SPAYED' in str(x) else str(x))

    # are multiple breeds listed?
    df['Is_Mixed_Breed'] = df.Breed.apply(lambda x: 1 if '/' in str(x) or '&' in str(x) or 'MIX' in str(x) else 0)

    # are multiple breeds listed?
    df['Is_Multicolor'] = df.Color.apply(lambda x: 1 if '/' in str(x) else 0)

    return df

def clean_df(df, params):
    '''
    clean_df handles null data, renames columns to be more friendly in python, drops duplicates and more!


    params options
    na_data will fill missing data with 'unknown', delete missing data or do nothing
    input options are...
        * 'fill'
        * 'drop'
        * 'nothing'
    drop_outlier_days removes pets who have a lenght of stay exceeding the value YOU enter
    input options are...
        * False
        * or any integer
    '''

    # remove space from col name, force strings to upper
    col_list = []
    for col in df.columns:
        new_col = col.replace(' ', '_')
        col_list.append(new_col)
        try:
            df[col] = df[col].str.upper()
        except:
            print(f'{col} is NOT A STRING')
    df.columns = col_list

    # drop Other/Bird/Livestock
    df = df[df.Type.isin(['DOG','CAT'])]
    # remove animals without an outcome or intake date
    df = df[~df.Outcome_Date.isnull()]
    df = df[~df.Intake_Date.isnull()]
    # Remove duplicates
    df.drop_duplicates(inplace=True)
    # Drop rows where 'Animal ID' is missing as it is a critical identifier
    df.dropna(subset=['Animal_ID'], inplace=True)
    df.drop(columns=['Count'], inplace=True)
    # update TORTIE to Tortoiseshell
    df.loc[df.Color=='TORTIE', 'Color'] = 'Tortoiseshell'.upper()

    # place nan token for remaining columns
    for col in df.columns:
        null_count = df[col][df[col].isnull()].shape[0]
        if null_count!= 0 and params['na_data'] != False:
            if params['na_data'].lower() == 'fill':
                print(f"replace null values in {col} with 'Unknown'")
                df[col].fillna('Unknown', inplace=True)

            elif params['na_data'].lower() == 'drop':
                print(f"drop null values in {col}")
                df.dropna(subset=[col], inplace=True)

    if params['drop_outlier_days'] != False:
        df = df[df.Days_in_Shelter < int(params['drop_outlier_days'])]

    return df

def train_validate_test_split(df, params):
    '''split data into train, validate, test'''

    stratify_col = params['sample_dict']['stratify_col']
    train_size = params['sample_dict']['train_size']
    validate_size = params['sample_dict']['validate_size']
    test_size = params['sample_dict']['test_size']
    random_state = 42

    assert train_size + validate_size + test_size == 1, "Train, validate, and test sizes must sum to 1."

    # Split data into train+validate and test
    train_validate, test = train_test_split(df, test_size=test_size, stratify=df[stratify_col], random_state=random_state)

    # Calculate the proportion of validate
    validate_prop = validate_size / (train_size + validate_size)

    # Split train+validate into train and validate
    train, validate = train_test_split(train_validate, test_size=validate_prop, stratify=train_validate[stratify_col], random_state=random_state)

    return train, validate, test

def load_glove_embeddings(file_path='glove.6B.50d.txt'):
    embeddings_index = {}
    with open(file_path, encoding="utf-8") as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index

def get_word_embedding(word, embeddings_index):
    word = word.lower()
    return embeddings_index.get(word, np.zeros(50))  # 50 is the dimension of the GloVe vectors

def get_mean_color_embedding(color, color_embeddings):
    color = color.replace('/',' ')
    colors = color.split(' ')
    embeddings = [color_embeddings[c] for c in colors if c in color_embeddings]
    return np.mean(embeddings, axis=0)

def get_mean_breed_embedding(breed, breed_embeddings):
    breed = breed.replace('/',' ')
    breed = breed.replace('&',' ')
    breed = breed.replace('   ',' ')
    breed = breed.replace('  ',' ')
    breeds = breed.split(' ')
    embeddings = [breed_embeddings[c] for c in breeds if c in breed_embeddings]
    return np.mean(embeddings, axis=0)

def embed_colors(df, embeddings_index):
    # Extract unique colors and get their embeddings
    unique_colors = df['Color'].str.replace('/', ' ').str.split(' ').explode().unique()
    color_embeddings = {color: get_word_embedding(color, embeddings_index) for color in unique_colors}
    if 'TORTIE' in color_embeddings.keys():
        color_embeddings['TORTIE'] = color_embeddings['Tortoiseshell'.upper()]
    # Apply the function to create embeddings for the 'Color' column
    df['Color_Embedding'] = df['Color'].apply(lambda x: get_mean_color_embedding(x, color_embeddings))
    # Perform PCA to reduce dimensionality to 2D
    pca = PCA(n_components=2)
    reduced_embeddings = pca.fit_transform(np.array(df.Color_Embedding.tolist()))
    # Apply KMeans clustering
    n_clusters = 5  # Define the number of clusters
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    df['Color_Embedding_Cluster'] = kmeans.fit_predict(reduced_embeddings)
    return df

def embed_breeds(df, embeddings_index):
    # Extract unique colors and get their embeddings
    unique_breeds = df['Breed'].str.replace('/',' ').str.replace('&',' ').str.split(' ').explode().unique()
    breed_embeddings = {breed: get_word_embedding(breed, embeddings_index) for breed in unique_breeds}
    df['Breed_Embedding'] = df['Breed'].apply(lambda x: get_mean_breed_embedding(x, breed_embeddings))
    # Perform PCA to reduce dimensionality to 2D
    pca = PCA(n_components=2)
    reduced_embeddings = pca.fit_transform(np.array(df.Breed_Embedding.tolist()))
    # Apply KMeans clustering
    n_clusters = 5  # Define the number of clusters
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    df['Breed_Embedding_Cluster'] = kmeans.fit_predict(reduced_embeddings)
    return df

def sklearn_pipeline(train_df,validate_df):
    # Define feature columns and target column
    feature_cols = ['Type', 'Sex', 'Size', 'Intake_Type', 'Intake_Subtype',
       'Intake_Condition', 'Multiple_Visit_Count',
       'Age_inDays_at_Outcome', 'Age_Group', 'Is_Aggressive', 'Has_Name',
       'Is_Fixed', 'Is_Mixed_Breed', 'Is_Multicolor', 'Color_Embedding_Cluster',
       'Breed_Embedding_Cluster']
    target_col = 'Days_in_Shelter_Label'

    # Split data into training and testing sets
    X_train = train_df[feature_cols]
    y_train = train_df[target_col]
    X_test = validate_df[feature_cols]
    y_test = validate_df[target_col]

    # Define the column transformer with OneHotEncoder for categorical columns and StandardScaler for numerical columns
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), ['Age_inDays_at_Outcome', 'Multiple_Visit_Count', 'Color_Embedding_Cluster',
                'Breed_Embedding_Cluster']),
            ('cat', OneHotEncoder(), ['Type', 'Sex', 'Size', 'Intake_Type', 'Intake_Subtype',
                'Intake_Condition','Age_Group', 'Is_Aggressive', 'Has_Name',
                'Is_Fixed', 'Is_Mixed_Breed', 'Is_Multicolor'])
        ])

    return preprocessor, X_train, y_train, X_test, y_test


if __name__ == '__main__':
    '''
    params options

    na_data will fill missing data with 'unknown', delete missing data or do nothing
    input options are...
        * 'fill'
        * 'drop'
        * False

    drop_outlier_days removes pets who have a lenght of stay exceeding the value YOU enter
    input options are...
        * False
        * or any integer

    embed creates 50x1 embedding vectors for color and breed
        download https://nlp.stanford.edu/data/glove.6B.zip, unzip and save in repo
        * True
        * False

    sample_dict controls stratified sampling
        * stratify_col: a column name used for stratified sampling... spelling and caps must be exact
        * train_size: a fraction of data you want for the training data
        * validate_size: a fraction of data you want for the validate data
        * test_size: a fraction of data you want for the test data

    num_buckets how many buckets to break up length of stay into for model training
        creates new column Days_in_Shelter_Label
        * input is a integer

    '''

    params = {
            'na_data': 'fill',
            'drop_outlier_days': 300,
            'embed':True,
            'num_buckets':3,
            'sample_dict':
                {
                'stratify_col':'Type',
                'train_size':0.6, 'validate_size':0.2, 'test_size':0.2
                }
            }
    train_df, validate_df, test_df = load_df(params)

<ipython-input-42-6399c308ebb2>:48: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df =  pd.read_csv(name)


Unnamed: 0 is NOT A STRING
Intake Date is NOT A STRING
Count is NOT A STRING
Outcome Date is NOT A STRING
Days in Shelter is NOT A STRING
Date Of Birth is NOT A STRING
replace null values in Name with 'Unknown'
replace null values in Outcome_Subtype with 'Unknown'


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
